# Importar librerías

In [1]:
# Imports estándar
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import datetime

# scikit-learn
from sklearn.dummy           import DummyClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.linear_model    import LinearRegression
from sklearn.linear_model    import Lasso
from sklearn.metrics         import confusion_matrix
from sklearn.metrics         import roc_curve
from sklearn.metrics         import roc_auc_score
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import train_test_split

# Definir funciones que nos serán útiles

In [2]:
def estandarizacion(X):
    """ 
    Función para estandarizar una variable
    
    Args:
        x     set de variables independientes
    """
    media_X = np.mean(X)
    X = X - media_X
    std_X = np.std(X)
    X = X / std_X
    return X, media_X, std_X

# Importar set de datos

In [3]:
# Importar datos
data = pd.read_csv('data/height_weight_genders.csv')

In [4]:
# Hecha un vistazo a los datos
data.head()

,Gender,Height,Weight
0,Male,73.847017,241.893563
1,Male,68.781904,162.310473
2,Male,74.110105,212.740856
3,Male,71.730978,220.042470
4,Male,69.881796,206.349801


# Preprocesa set de datos
Para este ejercicio no vamos a dividir el set en entrenamiento y validación, nos quedaremos con un solo set de datos

In [5]:
# Crear variables dummy para la variable categórica "Gender"
data = pd.get_dummies(data=data, columns=['Gender'])

In [6]:
# Mostrar el nombre de las variables dummy creadas
print('\n'.join(data.columns))

Height
Weight
Gender_Female
Gender_Male


In [7]:
# Hecha un vistazo a los datos
data.head()

,Height,Weight,Gender_Female,Gender_Male
0,73.847017,241.893563,0,1
1,68.781904,162.310473,0,1
2,74.110105,212.740856,0,1
3,71.730978,220.042470,0,1
4,69.881796,206.349801,0,1


In [8]:
# Define la variable target (Weight)
y = np.array(data['Weight'])

# Define el conjunto de variables independientes
atributos = list(set(list(data.columns)) - set(['Weight']))
X = data.loc[:, atributos]

In [9]:
# Checa el tamaño de las variables dependiente e independientes
y.shape, X.shape

((10000,), (10000, 3))

In [10]:
# Estandarizar las variables independientes
X, media_X, std_X = estandarizacion(X)

In [11]:
# Hecha un vistazo a las variables estandarizadas
X.head()

,Gender_Male,Gender_Female,Height
0,1.0,-1.0,1.944061
1,1.0,-1.0,0.627537
2,1.0,-1.0,2.012443
3,1.0,-1.0,1.394060
4,1.0,-1.0,0.913421


# Define las funciones que nos van a servir para ejecutar el algoritmo de gradiente descendente

## Función de costo error cuadrático medio (MSE)
 $\mathcal{L}(w) =  \displaystyle \frac{1}{N} \sum_{i=1}^{N} \textbf{e}^2$, $\textbf{e} = \textbf{y-Xw}$

In [12]:
def calcula_costo_mse(y, X, w):
    """ 
    Función para calcular el valor de la función de costo/pérdida
    
    Args:
        y     arreglo de numpy que contiene variable target
        X     matriz que contiene variables independientes
        w     arreglo de numpy que contiene parámetros w
    """
    e = y - X.dot(w)
    mse = np.mean(e**2)
    return mse

## Fórmula del gradiente de la función de costo error cuadrático medio (MSE)
$\nabla \mathcal{L}(w) =  \displaystyle -\frac{1}{N} \textbf{X}^T\textbf{e}$, $\textbf{e} = \textbf{y-Xw}$

In [13]:
def calcula_gradiente(y, X, w):
    """ 
    Función para calcular el valor del gradiente
    
    Args:
        y     arreglo de numpy que contiene variable target
        X     matriz que contiene variables independientes
        w     arreglo de numpy que contiene parámetros w
    """
    e = y - X.dot(w)
    gradiente = -X.T.dot(e) / len(e)
    return gradiente

## Pasos para llevar a cabo el algoritmo gradiente descendente
Itera las veces que sean necesarias
1. Calcula el valor de la función de costo para el vector w inicial
2. Calcula el valor del gradiente de la función de costo en ese valor w
3. Da un paso de tamaño $\gamma$ en dirección contraria al gradiente de la función de costo, i.e. $\textbf{w}^{(t+1)} = \textbf{w}^{(t)}-\gamma \nabla\mathcal{L}(\textbf{w}^{(t)})$
4. Guarda el valor de los parámetros ws y de los valores de la función de costo en cada iteración
5. Imprime resultados de cada iteración para que veas como va la ejecución del algoritmo 

In [14]:
def gradiente_descendente(y, X, w_inicial, max_iters, gamma):
    """ 
    Función para llevar a cabo el algoritmo de optimización llamado gradiente descendente
    
    Args:
        y             arreglo de numpy que contiene variable target
        X             matriz que contiene variables independientes
        w_inicial     arreglo de numpy que contiene parámetros w iniciales
        max_iters     número de iteraciones máximas para ejecutar el algoritmo
        gamma         parámetro de aprendizaje
    """
    # Define variables para guardar w y el valor de la función de costo
    ws = [w_inicial]
    costos = []
    w = w_inicial
    for n_iter in range(max_iters):
        # Calcula el valor de la función de costo y del gradiente
        costo = calcula_costo_mse(y, X, w)
        gradiente = calcula_gradiente(y, X, w)
        # Actualiza el valor de w
        w = w - gamma * gradiente
        # Almacena el valor de w y de la función de costo
        ws.append(w)
        costos.append(costo)
        print("Gradiente Descendente({bi}/{ti}): valor función de costo={l}, w0={w0}, w1={w1}, w2={w2}".format(
              bi=n_iter, ti=max_iters - 1, l=costo, w0=w[0], w1=w[1], w2=w[2]))

    return costos, ws

In [15]:
# Definir parámetros para iniciar con el algoritmo
max_iters = 50
gamma = 0.7

# Inicializar el vector de parámetros w
w_inicial = np.array([0,0,0])

# Iniciar algoritmo de gradiente descendente
inicio = datetime.datetime.now()
costo_gradiente, ws_gradiente = gradiente_descendente(y, X, w_inicial, max_iters, gamma)
final = datetime.datetime.now()

# Imprimir proceso de ejecución
tiempo_ejecucion = (inicio - final).total_seconds()
print("Gradiente Descendente: tiempo de ejecución={t:.3f} segundos".format(t=tiempo_ejecucion))

Gradiente Descendente(0/49): valor función de costo=27093.837574561636, w0=17.906184677753693, w1=-17.906184677753693, w2=20.783697586992925
Gradiente Descendente(1/49): valor función de costo=26770.71679162065, w0=0.6895882288757171, w1=-0.6895882288757171, w2=9.694557903357348
Gradiente Descendente(2/49): valor función de costo=26561.962555030197, w0=12.940602863009842, w1=-12.940602863009842, w2=23.024887757957828
Gradiente Descendente(3/49): valor función de costo=26426.096016492695, w0=1.591644253073607, w1=-1.591644253073607, w2=15.171119598834544
Gradiente Descendente(4/49): valor función de costo=26337.133067123337, w0=9.930491395856004, w1=-9.930491395856004, w2=23.795116293230404
Gradiente Descendente(5/49): valor función de costo=26278.597524419347, w0=2.423090457278823, w1=-2.423090457278823, w2=18.314473573199063
Gradiente Descendente(6/49): valor función de costo=26239.932416356834, w0=8.07731408755412, w1=-8.07731408755412, w2=23.93369757328183
Gradiente Descendente(7/49

# Confirma predicción utilizando el objeto de regresión lineal de sklearn

## Fórmula de regresión lineal con tres variables independientes
$y = b + w_0x_0 + w_1x_1 + w_2x_2$

In [16]:
# Entrena una regresión lineal para confirmar los resultados anteriores
modelo_LinReg = LinearRegression()
modelo_LinReg.fit(X, y)
y_pred_proba_LinReg = modelo_LinReg.predict(X)

In [17]:
# Revisa la medida de R^2 de la regresión anterior
modelo_LinReg.score(X,y)

0.9027481729220092

In [18]:
# Checa los coefientes del modelo
print("Los parámetros del modelo son: w0= {0}, w1={1}, w2={2}".format(modelo_LinReg.coef_[0], modelo_LinReg.coef_[1], modelo_LinReg.coef_[2]))

Los parámetros del modelo son: w0= 4.8444276304793, w1=-4.8444276304792995, w2=22.99529959926877
